# Time performance Benchmark

Here, we compare the time processing for computing F.R.C. by using the Python packages *GeneralisedFormanRicci*, *HodgeLaplacians* and *fastforman*.

To this purpose, we locally provided specific package versions in whitch those packages were tested:

- `GeneralisedFormanRicci:` This code computes the Forman Ricci Curvature for simplicial complex generated from a given point cloud data.
    
  Last Update: 9 of January 2021. Version: 0.3
  
  Link: https://github.com/ExpectozJJ/GeneralisedFormanRicci/
  
  
- `HodgeLaplacians:` This package provides an interface for construction of Hodge and Bochner Laplacian matrices from the set of simplices. We use `Gudhi` algorithm for generating the simpleces from point cloud data. Here, we modify the package input so that the computation can be done to restricted simplex dimensions. 

  Last Update: 2019
  
  Link: https://github.com/tsitsvero/hodgelaplacians
  
  
# Benchmark Results:

We generated point cloud data for comparing the time processing of between the algorithms in the literature. Here, we consider the total time processing - from computing the cliques to computing the FRC. We perform the process in function of the number of points and the distance threshold. To see more a detailed benchmark, visit:
https://www.kaggle.com/datasets/danillosouza2020/forman-ricci-curvature-benchmark-report

Importing packages:

In [153]:
from time import time
import numpy as np
import re
import sys
import networkx as nx
import pandas as pd
from itertools import combinations
from GeneralisedFormanRicci.frc import GeneralisedFormanRicci
from hodgelaplacians import HodgeLaplacians
import gudhi as gd
from math import dist
import fastforman as ff

Defining function for estimating time processing (in seconds):

In [149]:
#Compute cells (cliques) using Gudhi package:
def cliques_gudhi(data,f,d=2):
   
    skeleton = gd.RipsComplex(points = data, max_edge_length = f)
    Rips_simplex_tree_sample = skeleton.create_simplex_tree(max_dimension = d)
    rips_generator = Rips_simplex_tree_sample.get_filtration()

    L=[i[0] for i in rips_generator]

    return L


def run_GeneralisedFormanRicci(data,r):
   
    
    t0=time()
    #Computing simplicial complex:
    sc=GeneralisedFormanRicci(data,epsilon=r)
    #Computing FRC:
    output=sc.compute_forman()
    t1=time()
    print("Total time processing: "+str(t1-t0)+" seconds.")
    return t1-t0,output

def run_hodge(data,r,d=2):
    t0=time()
    #Computing simplicial complex
    C=cliques_gudhi(data,r,d)
        
    
    
    hl=HodgeLaplacians(C, maxdimension=d+1)
    F={}
    
    dims = [i for i in range(1,d+1)]
    #Computing FRC for each desired simplex dimension:
    for dim in dims:
            try:
                f=hl.getCombinatorialRicci(dim)
                F[dim]=f
            
              
            except:
                next
    
    t1=time()
    #   
    print("Total time processing: "+str(t1-t0)+" seconds")
    return t1-t0,F

In [122]:
time_GFR,GFR=run_GeneralisedFormanRicci(D,0.2)

Total time: 0.021702289581298828 seconds.


In [132]:
time_HL,HFR=run_hodge(D,0.2,2)

Total time processing: 0.0057811737060546875 seconds


In [145]:
np.mean(list(GFR[2].values()))

2.8333333333333335

Defining function for generating random data for test:

In [16]:
def graph_from_cloud(data,e=2):
    G=nx.Graph()
    G.add_nodes_from([i for i in range(len(data))])
    counter=0
    for pair in data:
        G.add_node(counter,pos=(pair[0],pair[1],pair[2]))
        counter+=1
    pos_dict=nx.get_node_attributes(G,"pos")
    
    for pair in combinations(list(G.nodes()),2):
        n,m=pair
        X=pos_dict[n]
        Y=pos_dict[m]
       
        d=dist(X,Y)
        if d<=e:
            G.add_edge(n,m)
            G[n][m]['weight']=d
    return G

Generating 3D point cloud Data, with n=50 nodes and threshold distance r=0.5:

In [68]:
n=70
r=0.5
D=[np.random.uniform(0,1,3) for i in range(n)]

# GeneralisedFormanRicci

Time for generating simplicial complex:

In [69]:
%time sc=GeneralisedFormanRicci(D,epsilon=r)

CPU times: user 8.42 s, sys: 5.07 s, total: 13.5 s
Wall time: 2.23 s


Time for computing FRC for edges and triangles:

In [77]:
%%capture captured_output
%time frc_generalised=sc.compute_forman()

# HodgeLaplacians